# Figure 2 main paper

In [2]:
# default print properties
multiplier = 2

pixel_cm_ration = 36.5

width_full = int(13.95 * pixel_cm_ration) * multiplier
width_half = int(13.95/2 * pixel_cm_ration) * multiplier
width_quarter = int(13.95/2 * pixel_cm_ration) * multiplier

height_default_1 = int(3.5 * pixel_cm_ration) * multiplier
height_half = int(3.5/2 * pixel_cm_ration) * multiplier

# margins in pixel
top_margin = 25 * multiplier 
left_margin = 0 * multiplier 
right_margin = 0 * multiplier 
bottom_margin = 30 * multiplier 

font_size = 10 * multiplier 
font_family='Times New Roman'

line_width = 2 * multiplier 

In [3]:
# Define and load data
import autodisc as ad
import ipywidgets
import plotly
import numpy as np
import collections
import os
import sys
plotly.offline.init_notebook_mode(connected=True)

data_filters = collections.OrderedDict()
data_filters['none'] = []
data_filters['SLP'] = ('classifier_animal.data', '==', True)
data_filters['TLP'] = (('classifier_dead.data', '==', False), 'and', ('classifier_animal.data', '==', False))

org_experiment_definitions = dict()

org_experiment_definitions['main_paper'] = [
    dict(id = '1',
         directory = '../experiments/IMGEP-BC-BetaVAE',
         name = 'IMGEP-BC-BetaVAE',
         is_default = True),
    dict(id = '2',
         directory = '../experiments/IMGEP-BC-PatchBetaVAE',
         name = 'IMGEP-BC-PatchBetaVAE',
         is_default = True),
    dict(id = '3',
         directory = '../experiments/IMGEP-BC-LeniaStatistics',
         name = 'IMGEP-BC-LeniaStatistics',
         is_default = True),
    dict(id = '4',
         directory = '../experiments/IMGEP-BC-EllipticalFourier',
         name = 'IMGEP-BC-EllipticalFourier',
         is_default = True),
    dict(id = '5',
         directory = '../experiments/IMGEP-BC-SpectrumFourier',
         name = 'IMGEP-BC-SpectrumFourier',
         is_default = True),
]
repetition_ids = list(range(10))

# define names and load the data
experiment_name_format = '<name>' # <id>, <name>

#global experiment_definitions
experiment_definitions = []
experiment_statistics = []

current_experiment_list = 'main_paper'

experiment_definitions = []
for org_exp_def in org_experiment_definitions[current_experiment_list]:
    new_exp_def = dict()
    new_exp_def['directory'] = org_exp_def['directory']
    if 'is_default' in org_exp_def:
        new_exp_def['is_default'] = org_exp_def['is_default']

    if 'name' in org_exp_def:
        new_exp_def['id'] = ad.gui.jupyter.misc.replace_str_from_dict(experiment_name_format, {'id': org_exp_def['id'], 'name': org_exp_def['name']})
    else:
        new_exp_def['id'] = ad.gui.jupyter.misc.replace_str_from_dict(experiment_name_format, {'id': org_exp_def['id']})

    experiment_definitions.append(new_exp_def)

experiment_statistics = dict()
for experiment_definition in experiment_definitions:
    experiment_statistics[experiment_definition['id']] = ad.gui.jupyter.misc.load_statistics(experiment_definition['directory'])
       

In [4]:
BC_bvae_analytic_space_ranges = dict()
BC_patchbvae_analytic_space_ranges = dict()
BC_leniastatistics_analytic_space_ranges = dict()
BC_ellipticalfourier_analytic_space_ranges = dict()
BC_spectrumfourier_analytic_space_ranges = dict()
for i in range(8):
    BC_bvae_analytic_space_ranges[('BC_bvae_analytic_space_representations','data','[{}]'.format(i))] = (0, 1)
    BC_patchbvae_analytic_space_ranges[('BC_patchbvae_analytic_space_representations','data','[{}]'.format(i))] = (0, 1)
    BC_leniastatistics_analytic_space_ranges[('BC_leniastatistics_analytic_space_representations','data','[{}]'.format(i))] = (0, 1)
    BC_ellipticalfourier_analytic_space_ranges[('BC_ellipticalfourier_analytic_space_representations','data','[{}]'.format(i))] = (0, 1)
    BC_spectrumfourier_analytic_space_ranges[('BC_spectrumfourier_analytic_space_representations','data','[{}]'.format(i))] = (0, 1)

In [5]:
default_config = dict(
    plotly_format = 'svg',
    scatter_type= 'scatter_polar', 

    layout = dict(
        font = dict(
            family=font_family, 
            size=font_size, 
            ),
        updatemenus=[],
        width=width_half, # in cm
        height=height_default_1, # in cm
        
        margin = dict(
            l=left_margin, #left margin in pixel
            r=right_margin, #right margin in pixel
            b=bottom_margin, #bottom margin in pixel
            t=top_margin,  #top margin in pixel
            ),

        legend=dict(
            xanchor='left',
            yanchor='top',
            y=1,
            x=1,
         ),        
        ),
    
    default_colors = ['rgb(204,121,167)', 
                      'rgb(0,114,178)',
                      'rgb(230,159,0)',  
                      'rgb(0,158,115)',
                      'rgb(0,0,0)',
                      'rgb(240,228,66)',
                      'rgb(213,94,0)',  
                      'rgb(86,180,233)',
                     'rgb(214,39,40)',
                     'rgb(148,103,189)',
                     'rgb(140,86,75)',
                     'rgb(127,127,127)'],

    
    default_mean_trace = dict(line=dict(width = line_width)),
    
    mean_traces = [
        dict(line = dict(dash = 'dot')),
        dict(line = dict(dash = 'dot')),
        dict(line = dict(dash = 'dash')),
        dict(line = dict(dash = 'longdashdot')),
        dict(line = dict(dash = 'solid')),
        dict(line = dict(dash = 'dashdot')),
        dict(line = dict(dash = 'longdashdot')),
        dict(line = dict(dash = 'solid')),
        dict(line = dict(dash = 'dash')),
        dict(line = dict(dash = 'dashdot')),
        dict(line = dict(dash = 'dot')),
        dict(line = dict(dash = 'longdash')),
        dict(line = dict(dash = 'longdashdot')),
    ],
   
)

In [6]:
# General Functions to load data
import warnings

def calc_number_explored_bins(vectors, data_filter_inds, bin_config, ignore_out_of_range_values=False):
    number_explored_bins_per_step = []
    
    # create section borders
    bins_per_dim = []
    for dim_config in bin_config:
         bins_per_dim.append(np.linspace(dim_config[0], dim_config[1], num=dim_config[2]+1))

    # identify point for every vector
    count_per_section = collections.defaultdict(int)

    for vector in vectors:

        section = []

        # check each dimension
        for dim_idx in range(len(vector)):

            # identify at which section in de fined grid the value falls
            idxs = np.where(bins_per_dim[dim_idx] > vector[dim_idx])[0]

            if len(idxs) == 0:
                # value is larger than upper grid border
                #warnings.warn('A Vector with value {} is outside the defined grid for dimension {}.'.format(vector[dim_idx], dim_idx))

                if ignore_out_of_range_values:
                    section = None
                    break
                else:
                    section_idx = len(bins_per_dim[dim_idx])

            elif idxs[0] == 0:
                # value is smaller than lower grid border
                #warnings.warn('A Vector with value {} is outside the defined grid for dimension {}.'.format(vector[dim_idx], dim_idx))

                if ignore_out_of_range_values:
                    section = None
                    break
                else:
                    section_idx = -1
            else:
                section_idx = idxs[0]-1

            section.append(section_idx)

        if section is not None:
            section = tuple(section)
            count_per_section[section] += 1
            
        cur_n_bins = len(count_per_section)
        
        number_explored_bins_per_step.append(cur_n_bins)
                
    return number_explored_bins_per_step[-1]


def calc_number_explored_bins_for_experiments(experiment_definitions, source_data, space_definitions, ignore_out_of_range_values=False, data_filter=None):
    
    data_filter_inds = None
    if data_filter is not None and data_filter:
        # filter data according data_filter the given filter
        data_filter_inds = ad.gui.jupyter.misc.filter_experiments_data(source_data, data_filter)
    
    data_number_explored_bins_per_exp = dict()

    
    data_diversity = dict()
    space_names = [list(i.keys())[0][0] for i in space_definitions]
    
    for exp_def in experiment_definitions:
        exp_id = exp_def['id']
        
        rep_data_matrices = dict()
        for space_name in space_names:
            rep_data_matrices[space_name] = []

        
        bin_configs = []
        cur_data_filter_inds = data_filter_inds[exp_id] if data_filter_inds is not None else None  

        # load data and define the bin_config
        for space_idx in range(len(space_definitions)):
            space_name = space_names[space_idx]
            space_definition = space_definitions[space_idx]
            cur_bin_config = []
            for dim_name, dim_ranges in space_definition.items():

                # define the bin configuration for the current parameter
                cur_bin_config.append((dim_ranges[0], dim_ranges[1], 10))

                # get all repetition data for the current paramter
                cur_data = ad.gui.jupyter.misc.get_experiment_data(data=source_data, experiment_id=exp_id, data_source=dim_name, repetition_ids='all', data_filter_inds=cur_data_filter_inds)
                # go over repetitions
                for rep_idx, cur_rep_data in enumerate(cur_data):
                    if rep_idx not in repetition_ids:
                        break;
                    cur_rep_data = np.array([cur_rep_data]).transpose()

                    if rep_idx >= len(rep_data_matrices[space_name]):
                        rep_data_matrices[space_name].append(cur_rep_data)
                    else:
                        rep_data_matrices[space_name][rep_idx] = np.hstack((rep_data_matrices[space_name][rep_idx], cur_rep_data))
            bin_configs.append(cur_bin_config)
            
        data_diversity[exp_id] = dict()
        cur_diversity_data = np.zeros((len(cur_data), len(space_definitions))) #n_repet*n_spaces
        for space_idx in range(len(space_definitions)):
            space_name = space_names[space_idx]
            space_definition = space_definitions[space_idx]
            for rep_idx, rep_matrix_data in enumerate(rep_data_matrices[space_name]):
                cur_rep_data_filter_inds = data_filter_inds[exp_id][rep_idx] if cur_data_filter_inds is not None else None  
                rep_data =calc_number_explored_bins(rep_matrix_data, cur_rep_data_filter_inds, bin_configs[space_idx], ignore_out_of_range_values=ignore_out_of_range_values)
                cur_diversity_data[rep_idx, space_idx] = rep_data
        
            
            data_diversity[exp_id]['n_explored_bins'] = cur_diversity_data

    return data_diversity


In [7]:
data_filter_name = 'none'
data_filter = data_filters[data_filter_name]    
# Load data
BC_spaces_ranges = [BC_bvae_analytic_space_ranges,
                    BC_patchbvae_analytic_space_ranges,
                    BC_leniastatistics_analytic_space_ranges,
                    BC_ellipticalfourier_analytic_space_ranges,
                    BC_spectrumfourier_analytic_space_ranges,
                   ]

data_diversity = calc_number_explored_bins_for_experiments(
    experiment_definitions, 
    experiment_statistics, 
    BC_spaces_ranges, 
    data_filter=data_filter)

In [8]:
# Normalization by the maximum along each axis
data_diversity_copy = data_diversity
max_per_BC = np.zeros(len(BC_spaces_ranges))
for exp_def in experiment_definitions:
    exp_id = exp_def['id']
    max_per_BC = np.maximum(max_per_BC, data_diversity[exp_id]['n_explored_bins'].max(0))
for exp_def in experiment_definitions:
    exp_id = exp_def['id']
    data_diversity[exp_id]['n_explored_bins'] /= max_per_BC

In [10]:
# PLOTTING
config = default_config
config['layout']['title'] = dict(text='Diversities of patterns in each BC', xanchor='left', x=0.04, yanchor='top', y=1)
config['layout']['legend'] = dict(xanchor='right', x=1, yanchor='bottom', y=0)
config['layout']['width'] = width_full
config['layout']['height'] = height_default_1 * 2
config['thetas'] =  ['BetaVAE','Patch-BetaVAE','Lenia-Statistics','Elliptical-Fourier','Spectrum-Fourier']
fig = ad.gui.jupyter.plot_scatter_per_datasource(
        experiment_ids=[exp_def['id'] for exp_def in experiment_definitions],
        repetition_ids=repetition_ids, 
        data=data_diversity, 
        data_source=['n_explored_bins'], 
        config=config)   

#plotly.io.write_image(fig, 'main_figure_2.pdf')

/home/mayalen/miniconda3/envs/holmes/lib/python3.6/site-packages/plotly/tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead

